In [22]:
from models.PolatoAdaBoost import polato_AdaBoost_eval
from models.FRF import FRF_eval
from models.FL_AdaBoost_Dist import FLEnsembleDist
from sklearn.model_selection import train_test_split
from flextrees.datasets.tabular_datasets import adult
import numpy as np

Load the dataset

In [26]:
seed = 0
train_data, test_data = adult(ret_feature_names=False, categorical=False)
X_data,y_data = train_data.to_numpy()
X_test,y_test = test_data.to_numpy()
X_data = np.concatenate((X_data,X_test))
y_data = np.concatenate((y_data,y_test))

In [27]:
#Separate public data
data,public_data,targets,predicted_public_data = train_test_split(X_data,y_data,test_size=0.25,train_size=0.75,random_state=seed)
#Get train and test set
X_train,X_test,y_train,y_test = train_test_split(data,targets,test_size=0.1,train_size=0.9,random_state=seed)

In [28]:
#When creating the model the data partition according to the chosen data distribution is made. 
o_s_Wmv_cw = FLEnsembleDist(X_train,y_train,public_data,Nclients=10,max_depth=8,T=10,
                                        data_distribution='niid_quantity_skew',distribution_param=0.5,
                                        public_data_prediction='weighted_majority_voting',server_alpha_weight_adj='common_weighted',
                                        prediction_weights='only_server',random_state=seed)
#store data distrib for later models
train_dict = o_s_Wmv_cw.train_clients_data.copy()
test_dict = o_s_Wmv_cw.test_clients_data.copy()
#Takes the columns of weights of AdaBoost out
for key,(train,labeltr) in train_dict.items():
    train_dict[key] = (train[:,:-1],labeltr)
    test,labelte = test_dict[key]
    test_dict[key] = (test[:,:-1],labelte)

In [29]:
#Train the model
o_s_Wmv_cw.fitmodel()

In [30]:
#We can simply evaluate how the federated model works in local tests (on average) and global test 
os_cw_acc_global,os_cw_f1_global,os_cw_acc_local,os_cw_f1_local = o_s_Wmv_cw.sumar_overall_score(X_test,y_test)
print(os_cw_acc_global,os_cw_f1_global,os_cw_acc_local,os_cw_f1_local)

84.19328419328419 85.08712268545413 86.50632468760037 87.33632926329648


In [31]:
#If we want to compare it to the local models, we first need to train local models with same data
o_s_Wmv_cw.fit_local_clients_models()

In [32]:
#Then we can get a Dataframe with all the results for each client
o_s_Wmv_cw.overall_acc_score(X_test,y_test)

,data_distrib,FL_acc_own_data,FL_acc_global_data,local_acc_own_data,local_acc_global_data,local_difference,global_difference
0,1202.0,81.343284,84.193284,79.104478,82.841933,2.238806,1.351351
1,55.0,100.000000,84.193284,71.428571,79.770680,28.571429,4.422604
2,596.0,82.089552,84.193284,77.611940,81.326781,4.477612,2.866503
3,1503.0,83.832335,84.193284,85.628743,82.473382,-1.796407,1.719902
4,162.0,89.473684,84.193284,84.210526,80.917281,5.263158,3.276003
5,487.0,85.454545,84.193284,81.818182,82.309582,3.636364,1.883702
6,7017.0,85.512821,84.193284,85.641026,83.988534,-0.128205,0.204750
7,5527.0,82.439024,84.193284,83.739837,83.988534,-1.300813,0.204750
8,1909.0,84.507042,84.193284,78.873239,82.022932,5.633803,2.170352
9,1314.0,90.410959,84.193284,87.671233,82.227682,2.739726,1.965602


In [33]:
#Now we can use it to compare it to other state of the art algorithm with the function:
#For FRF 100 estimators:
FRF_acc_global,FRF_f1_global,FRF_acc_local,FRF_f1_local = FRF_eval(train_dict,test_dict,X_test,y_test,hyperparameters='theirs')


In [34]:
#For DistBoost.F:
DB_acc_global,DB_f1_global,DB_acc_local,DB_f1_local = polato_AdaBoost_eval(train_dict,test_dict,
                                                                            X_test,y_test, seed, model='distsamme',n_estimators=10,max_depth=8) 


In [35]:
#PreWeak.F
PW_acc_global,PW_f1_global,PW_acc_local,PW_f1_local= polato_AdaBoost_eval(train_dict,test_dict, 
                                                                          X_test,y_test, seed, model='preweaksamme',n_estimators=10,max_depth=8)


In [36]:
#AdaBoost.F
AB_acc_global,AB_f1_global,AB_acc_local,AB_f1_local = polato_AdaBoost_eval(train_dict,test_dict,
                                                                            X_test,y_test, seed, model='adaboost.f1',n_estimators=10,max_depth=8)
            

In [37]:
print('FRF: ',FRF_acc_global,FRF_f1_global,FRF_acc_local,FRF_f1_local)
print('DistBoost.F: ',DB_acc_global,DB_f1_global,DB_acc_local,DB_f1_local)
print('PreWeak.F: ', PW_acc_global,PW_f1_global,PW_acc_local,PW_f1_local)
print('AdaBoost.F: ', AB_acc_global,AB_f1_global,AB_acc_local,AB_f1_local)

FRF:  83.33333333333334 81.69952769841449 82.66393359923019 80.75650513185138
DistBoost.F:  84.6027846027846 84.07257202689499 86.73970608896585 86.51509739585578
PreWeak.F:  85.0941850941851 84.42212597838255 85.43273774170373 84.7648646315737
AdaBoost.F:  84.35708435708436 83.875014868358 87.29246144230515 87.00041923813072
